# Примеры использования модулей

Этот notebook демонстрирует использование модулей:
- `maps.py` - работа с геокодированием и маршрутами
- `openweather.py` - получение данных о погоде
- `websearch.py` - поиск в интернете


In [1]:
# Импорты
import sys
from pathlib import Path

# Добавляем корневую директорию проекта в путь
project_root = Path.cwd().parent if Path.cwd().name == 'main' else Path.cwd()
sys.path.insert(0, str(project_root))

# Импортируем модули из папки main
from main.maps import YandexGeocoder, SimpleRouteEstimator, YandexRouteService
from main.openweather import OpenWeatherClient
from main.websearch import TavilyHtmlFetcher


## 1. Работа с картами (maps.py)

### Геокодирование адресов


In [9]:
# Создаем геокодер
geocoder = YandexGeocoder()

# Преобразуем адрес в координаты
address = "Москва, Красная площадь"
coords = geocoder.adress_to_geopoint(address)

if coords:
    lon, lat = coords
    print(f"Адрес: {address}")
    print(f"Координаты: долгота={lon}, широта={lat}")
else:
    print("Адрес не найден")


Адрес: Москва, Красная площадь
Координаты: долгота=37.621202, широта=55.753544


### Оценка маршрута между двумя точками


In [10]:
# Создаем оценщик маршрутов
estimator = SimpleRouteEstimator()

# Координаты двух точек (Санкт-Петербург)
start_point = (30.315868, 59.939095)  # Эрмитаж
end_point = (30.360909, 59.934280)    # Петропавловская крепость

# Оцениваем маршрут для разных режимов
route_info = estimator.estimate_route(start_point, end_point, modes=("walking", "car", "bus"))

print(f"Прямое расстояние: {route_info['distance_km_straight']:.2f} км")
print("\nОценка для разных режимов:")
for mode, info in route_info['modes'].items():
    print(f"\n{mode.upper()}:")
    print(f"  Расстояние: {info['distance_km']:.2f} км")
    print(f"  Время в пути: {info['duration_min']:.1f} минут")
    print(f"  Скорость: {info['speed_kmh']} км/ч")


Прямое расстояние: 2.57 км

Оценка для разных режимов:

WALKING:
  Расстояние: 4.49 км
  Время в пути: 53.9 минут
  Скорость: 5.0 км/ч

CAR:
  Расстояние: 4.62 км
  Время в пути: 6.9 минут
  Скорость: 40.0 км/ч

BUS:
  Расстояние: 5.13 км
  Время в пути: 12.3 минут
  Скорость: 25.0 км/ч


### Высокоуровневый сервис маршрутов (по адресам)


In [11]:
# Создаем сервис маршрутов
route_service = YandexRouteService(geocoder=YandexGeocoder())

# Получаем маршрут между адресами
from_addr = "Москва, Красная площадь"
to_addr = "Москва, ВДНХ"

try:
    route = route_service.route_by_addresses(from_addr, to_addr, modes=("walking", "car"))
    
    print(f"Маршрут: {route['from_address']} → {route['to_address']}")
    print(f"От точки: {route['from_point']}")
    print(f"До точки: {route['to_point']}")
    print(f"\nПрямое расстояние: {route['distance_km_straight']:.2f} км")
    
    for mode, info in route['modes'].items():
        print(f"\n{mode.upper()}:")
        print(f"  Расстояние: {info['distance_km']:.2f} км")
        print(f"  Время: {info['duration_min']:.1f} минут")
        
except ValueError as e:
    print(f"Ошибка: {e}")


Маршрут: Москва, Красная площадь → Москва, ВДНХ
От точки: (37.621202, 55.753544)
До точки: (37.641045, 55.821097)

Прямое расстояние: 7.61 км

WALKING:
  Расстояние: 13.32 км
  Время: 159.9 минут

CAR:
  Расстояние: 13.70 км
  Время: 20.6 минут


## 2. Работа с погодой (openweather.py)


In [5]:
# Создаем клиент для получения погоды
try:
    weather_client = OpenWeatherClient()
    
    # Получаем погоду для Москвы (координаты)
    lat, lon = 55.7558, 37.6173
    weather = weather_client.get_weather(lat, lon)
    
    print(f"Погода в Москве:")
    print(f"  Описание: {weather.description}")
    print(f"  Температура: {weather.temperature}°C")
    print(f"  Ощущается как: {weather.feels_like}°C")
    print(f"  Влажность: {weather.humidity}%")
    print(f"  Скорость ветра: {weather.wind_speed} м/с")
    
except RuntimeError as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что установлена переменная окружения OPENWEATHER_API_KEY")


Погода в Москве:
  Описание: overcast clouds
  Температура: 0.27°C
  Ощущается как: -3.86°C
  Влажность: 69%
  Скорость ветра: 3.87 м/с


### Получение погоды для координат из геокодера


In [6]:
# Комбинируем геокодер и погоду
try:
    # Получаем координаты адреса
    address = "Санкт-Петербург, Невский проспект"
    coords = geocoder.adress_to_geopoint(address)
    
    if coords:
        lon, lat = coords
        print(f"Адрес: {address}")
        print(f"Координаты: {lat}, {lon}\n")
        
        # Получаем погоду для этих координат
        weather = weather_client.get_weather(lat, lon)
        print(f"Погода:")
        print(f"  {weather.description}")
        print(f"  Температура: {weather.temperature}°C")
        print(f"  Влажность: {weather.humidity}%")
    else:
        print("Адрес не найден")
        
except RuntimeError as e:
    print(f"Ошибка: {e}")
except Exception as e:
    print(f"Ошибка: {e}")


Ошибка: name 'geocoder' is not defined


## 3. Поиск в интернете (websearch.py)


In [7]:
# Создаем поисковик
try:
    searcher = TavilyHtmlFetcher(default_max_results=3)
    
    # Ищем информацию
    query = "достопримечательности Санкт-Петербурга"
    pages = searcher.search(query, max_results=3)
    
    print(f"Результаты поиска по запросу: '{query}'\n")
    print(f"Найдено страниц: {len(pages)}\n")
    
    for i, page in enumerate(pages, 1):
        print(f"{i}. {page.title}")
        print(f"   URL: {page.url}")
        print(f"   HTML длина: {len(page.html)} символов")
        print()
        
except RuntimeError as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что установлена переменная окружения TAVILY_API_KEY")


Результаты поиска по запросу: 'достопримечательности Санкт-Петербурга'

Найдено страниц: 3

1. Главные достопримечательности Санкт-Петербурга
   URL: https://sokroma.ru/articles/chto-posetit/glavnye-dostoprimechatelnosti-sankt-peterburga/
   HTML длина: 106330 символов

2. Достопримечательности Санкт-Петербурга
   URL: https://www.sputnik8.com/ru/st-petersburg/sights
   HTML длина: 1144101 символов

3. Топ-100 достопримечательностей Санкт-Петербурга и ...
   URL: https://extraguide.ru/russia/saint-petersburg/sights/
   HTML длина: 394472 символов



### Пример поиска информации о погоде в конкретном месте


In [8]:
# Ищем информацию о погоде в определенном городе
try:
    city = "Сочи"
    query = f"погода {city} сегодня"
    pages = searcher.search(query, max_results=2)
    
    print(f"Поиск: '{query}'\n")
    for i, page in enumerate(pages, 1):
        print(f"{i}. {page.title}")
        print(f"   {page.url}")
        # Можно посмотреть первые 200 символов HTML
        print(f"   HTML превью: {page.html[:200]}...")
        print()
        
except Exception as e:
    print(f"Ошибка: {e}")


Поиск: 'погода Сочи сегодня'

1. Погода в Сочи сегодня - Gismeteo
   https://www.gismeteo.ru/weather-sochi-5233/
   HTML превью: ...

2. Погода в Сочи сейчас - Краснодарский край - Gismeteo
   https://www.gismeteo.ru/weather-sochi-5233/now/
   HTML превью: ...



## 4. Комбинированный пример

Пример использования всех модулей вместе для планирования поездки


In [ ]:
# Планируем поездку: получаем маршрут и погоду в пункте назначения
try:
    # 1. Получаем маршрут
    from_addr = "Москва, Красная площадь"
    to_addr = "Санкт-Петербург, Эрмитаж"
    
    route = route_service.route_by_addresses(from_addr, to_addr, modes=("car",))
    
    print("=" * 50)
    print("ПЛАНИРОВАНИЕ ПОЕЗДКИ")
    print("=" * 50)
    print(f"\nМаршрут: {from_addr} → {to_addr}")
    
    car_info = route['modes']['car']
    print(f"\nНа машине:")
    print(f"  Расстояние: {car_info['distance_km']:.1f} км")
    print(f"  Время в пути: {car_info['duration_h']:.1f} часов")
    
    # 2. Получаем погоду в пункте назначения
    to_lat, to_lon = route['to_point'][1], route['to_point'][0]
    weather = weather_client.get_weather(to_lat, to_lon)
    
    print(f"\nПогода в пункте назначения:")
    print(f"  {weather.description}")
    print(f"  Температура: {weather.temperature}°C")
    print(f"  Ощущается как: {weather.feels_like}°C")
    print(f"  Влажность: {weather.humidity}%")
    
    # 3. Ищем информацию о достопримечательностях
    print(f"\nПоиск информации о достопримечательностях...")
    search_query = f"достопримечательности {to_addr.split(',')[0]}"
    pages = searcher.search(search_query, max_results=2)
    
    if pages:
        print(f"\nНайдено {len(pages)} страниц:")
        for page in pages:
            print(f"  - {page.title}")
    
except Exception as e:
    print(f"Ошибка: {e}")
    import traceback
    traceback.print_exc()
